Silver


In [2]:
from datetime import date, timedelta
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType


#One week ago
#start_date = date.today() - timedelta(7)

StatementMeta(, 051590a0-8f1a-4757-ac14-c4ff8075a883, 4, Finished, Available, Finished)

In [7]:
#Lire le dataframe
df = spark.read.option("multiline", "true").json(f"Files/{start_date}_earthquake_data.json")
# Reshape earthquake data by extracting and renaming key attributes for further analysis.
df = \
df.\
    select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
        )
#display(df.take(1))

StatementMeta(, 051590a0-8f1a-4757-ac14-c4ff8075a883, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 94937ed2-7fc1-4f78-b47b-3290e271a50e)

In [8]:
# Convert 'time' and 'updated' columns from milliseconds to timestamp format for clearer datetime representation.
df = df.\
    withColumn('time', col('time')/1000).\
    withColumn('updated', col('updated')/1000).\
    withColumn('time', col('time').cast(TimestampType())).\
    withColumn('updated', col('updated').cast(TimestampType()))

StatementMeta(, 051590a0-8f1a-4757-ac14-c4ff8075a883, 10, Finished, Available, Finished)

In [9]:
# appending the data to the gold table
df.write.mode('append').saveAsTable('earthquake_events_silver')

StatementMeta(, 051590a0-8f1a-4757-ac14-c4ff8075a883, 11, Finished, Available, Finished)